In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
from helpers import DataExplorator

data_explorator = DataExplorator()

# dataset

the dataset provides training and validation images separately

In [ ]:
data_train = data_explorator.get_train_images()
data_train

In [ ]:
data_valid = data_explorator.get_valid_images()
data_valid

In [38]:
def histogram(img):
    #initializing
    WB = np.zeros(256)
    WG = np.zeros(256)
    WR = np.zeros(256)
    
    l, c, z = img.shape
    
    B = img[:,:,0]
    G = img[:,:,1]
    R = img[:,:,2]
    
    #count pixels by channel
    for i in range(l):
        for j in range(c):
            WB[B[i,j]] = WB[B[i,j]]+1
            WG[G[i,j]] = WG[G[i,j]]+1
            WR[R[i,j]] = WR[R[i,j]]+1
    
    #normalizing
    for i in range(256):
        WB[i] = WB[i]/(l*c)
        WG[i] = WG[i]/(l*c)
        WR[i] = WR[i]/(l*c)

    return [WB,WG,WR]

In [39]:
def extract_features(img):
    features = []
    hist = histogram(img)

    soma_B = 0
    soma_G = 0
    soma_R = 0

    for i in range(256):
        if((i+1)%64 == 0):
            features.append(soma_B)
            features.append(soma_G)
            features.append(soma_R)
            soma_B = 0
            soma_G = 0
            soma_R = 0
        else:
            soma_B+=hist[0][i]
            soma_G+=hist[1][i]
            soma_R+=hist[2][i]
    
    return features

In [ ]:
classes_codes = {"healthy": 0, "black_measles": 1, "leaf_blight": 2, "black_rot": 3}

In [ ]:
def get_features(img_path, img_class):
    image = cv2.imread(img_path)
    features = extract_features(image)
    
    features_dict = dict(zip(["feature_"+str(i+1) for i in range(len(features))], features))
    features_dict["class"] = classes_codes[img_class]
    return features_dict

In [ ]:
data_train_size = data_train.shape[0]

In [ ]:
%time data_train_features = [get_features(img, img_class) for img, img_class in zip(data_train['image'][:10], data_train['class'][:10])]

In [ ]:
data_train_df = pd.DataFrame(data_train_features)
train_data_df

In [ ]:
now_formated = datetime.now().strftime("%Y_%m_%d_%H_%M")
train_filename = "train_{}.csv".format(now_formated)
data_train_df.to_csv(train_filename, index=False)